In [1]:
# 10 by 10 meter Raterization for diff DEM labels. Labels from Kathrin Maier are appended 

In [2]:
from osgeo import gdal
from shapely.geometry import Polygon
import geopandas as gpd
import rasterio
from rasterio.features import shapes
import numpy as np
import pandas as pd

In [3]:
def rasterize_geojson(input_file, output_file, raster_extent): # Add raster_extent
    gdal.Rasterize(output_file, input_file, burnValues=[1], outputBounds=raster_extent, # add outputBounds
                   outputType=gdal.GDT_Byte, creationOptions=["COMPRESS=DEFLATE"], noData=255, initValues=0, xRes=10, yRes=-10, allTouched=True)

def polygonize_raster(input_file, output_file, year_start, year_end):
    with rasterio.open(input_file) as src:
        image = src.read(1)
        transform = src.transform
        polygons = []
        for shape, value in shapes(image, transform=transform):
            if value == 1.0:
                polygons.append(Polygon(tuple(shape['coordinates'][0])))
        gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:3413")
        gdf["year_start"] = year_start
        gdf["year_end"] = year_end
        gdf["type"] = ""
        gdf.to_file(output_file, driver='GeoJSON', crs="EPSG:3413")
        src.close()

        
def rasterize_labels(DEM_all, filepath, area_name, year_start, raster_extent): # add raster_extent
    years_end = DEM_all["year_end"].unique()

    for year in years_end:

        DEM_year = DEM_all[DEM_all["year_end"] == year]
        DEM_year.to_file(f'{filepath}_DEM_labels_noe_{year}.geojson', driver='GeoJSON')

        rasterize_geojson(f'{filepath}_DEM_labels_noe_{year}.geojson', f'{filepath}_temp_rast.tif', raster_extent) # add raster_extent

        input_file = f'{filepath}_temp_rast.tif'
        output_file = f'{filepath}_DEM_labels_noe_{year}.geojson'
        polygonize_raster(input_file, output_file, year_start, year)

    # add multiple geojson files together

    #create empty geodataframe
    dfs = []

    for year in years_end:
        DEM_year = gpd.read_file(f'{filepath}_DEM_labels_noe_{year}.geojson')
        dfs.append(DEM_year)

    merged_gdf = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs=DEM_all.crs)
    merged_gdf = merged_gdf.drop_duplicates(subset=['geometry'])
    merged_gdf.to_file(f'labels\\DEM_labels\\{area_name}_DEM_labels_rasterized.geojson', driver='GeoJSON', crs="EPSG:3413")

    import os
    os.remove(f'{filepath}_temp_rast.tif')

In [4]:
# Herschel
tile_gdf = gpd.read_file(f"boundries\\tiles\\tiles_herschel.geojson", crs="EPSG:3413")
raster_extent_herschel = tile_gdf.total_bounds
h_DEM_all = gpd.read_file('labels\\DEM_labels\\herschel_DEM_labels_noe.geojson', crs="EPSG:3413")
rasterize_labels(h_DEM_all, 'labels\\DEM_labels\\rasterized_and_polygonized\\herschel\\herschel', 'herschel', 2010, raster_extent_herschel)

# gydan
tile_gdf = gpd.read_file(f"boundries\\tiles\\tiles_gydan.geojson", crs="EPSG:3413")
raster_extent_gydan = tile_gdf.total_bounds
g_DEM_all = gpd.read_file('labels\\DEM_labels\\gydan_DEM_labels_noe.geojson', crs="EPSG:3413")
rasterize_labels(g_DEM_all, 'labels\\DEM_labels\\rasterized_and_polygonized\\gydan\\gydan', 'gydan', 2010, raster_extent_gydan)

# kolguev
tile_gdf = gpd.read_file(f"boundries\\tiles\\tiles_kolguev.geojson", crs="EPSG:3413")
raster_extent_kolguev = tile_gdf.total_bounds
k_DEM_all = gpd.read_file('labels\\DEM_labels\\kolguev_DEM_labels_noe.geojson', crs="EPSG:3413")
rasterize_labels(k_DEM_all, 'labels\\DEM_labels\\rasterized_and_polygonized\\kolguev\\kolguev', 'kolguev', 2010, raster_extent_kolguev)

# peel1
tile_gdf = gpd.read_file(f"boundries\\tiles\\tiles_peel.geojson", crs="EPSG:3413")
raster_extent_peel = tile_gdf.total_bounds
p_DEM_all = gpd.read_file('labels\\DEM_labels\\peel_DEM_labels_2013_2021.geojson', crs="EPSG:3413")
rasterize_labels(p_DEM_all, 'labels\\DEM_labels\\rasterized_and_polygonized\\peel\\peel', 'peel', 2013, raster_extent_peel)

c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [5]:
# Rasterization of Kathrin Maier's labels
k_gydan_DEM_labes = gpd.read_file('labels\\DEM_labels\\kathrin\\gydan_DEM_labels.geojson')
rasterize_labels(k_gydan_DEM_labes, 'labels\\DEM_labels\\kathrin\\gydan', 'kathrin_gydan', 2010,raster_extent_gydan)

k_peel_DEM_labes = gpd.read_file('labels\\DEM_labels\\kathrin\\peel_DEM_labels.geojson')
k_peel_DEM_labes_2010 = k_peel_DEM_labes[k_peel_DEM_labes["year_start"] == "2010"]
rasterize_labels(k_peel_DEM_labes_2010, 'labels\\DEM_labels\\kathrin\\peel', 'kathrin_peel10', 2010,raster_extent_peel)

k_peel_DEM_labes_2011 = k_peel_DEM_labes[k_peel_DEM_labes["year_start"] == '2011']
rasterize_labels(k_peel_DEM_labes_2011, 'labels\\DEM_labels\\kathrin\\peel', 'kathrin_peel11', 2011,raster_extent_peel)

In [6]:
import geopandas as gpd

# Read Kathrin Maier's Peel Labels

kathrin_gdf = gpd.read_file('labels\\DEM_labels\\kathrin_peel10_DEM_labels_rasterized.geojson')
kathrin_gdf2 = gpd.read_file('labels\\DEM_labels\\kathrin_peel11_DEM_labels_rasterized.geojson')

# Concatenate the two GeoDataFrames
kathrin_gdf = gpd.GeoDataFrame(pd.concat([kathrin_gdf, kathrin_gdf2], ignore_index=True), crs=kathrin_gdf.crs)
kathrin_gdf.to_file('labels\\DEM_labels\\k_peel_DEM_labels_rasterized.geojson', driver='GeoJSON')


In [ ]:

# Append 

# Peel
k_peel_DEM_labes = gpd.read_file('labels\\DEM_labels\\k_peel_DEM_labels_rasterized.geojson')
peel_DEM_labels_rasterized = gpd.read_file('labels/DEM_labels/peel_DEM_labels_rasterized.geojson')

# add together
peel_DEM_labels_rasterized = gpd.GeoDataFrame(pd.concat([peel_DEM_labels_rasterized, k_peel_DEM_labes], ignore_index=True), crs=peel_DEM_labels_rasterized.crs)

# gydan
k_gydan_DEM_labes = gpd.read_file('labels\\DEM_labels\\kathrin_gydan_DEM_labels_rasterized.geojson')
gydan_DEM_labels_rasterized = gpd.read_file('labels/DEM_labels/gydan_DEM_labels_rasterized.geojson')

# add together
gydan_DEM_labels_rasterized = gpd.GeoDataFrame(pd.concat([k_gydan_DEM_labes, gydan_DEM_labels_rasterized], ignore_index=True), crs=peel_DEM_labels_rasterized.crs)


# save
peel_DEM_labels_rasterized.to_file('labels/DEM_labels/peel_DEM_labels_rasterized.geojson', driver='GeoJSON')
gydan_DEM_labels_rasterized.to_file('labels/DEM_labels/gydan_DEM_labels_rasterized.geojson', driver='GeoJSON')
